# Imports

In [ ]:
import sys
sys.path.append('../')
from datetime import datetime
import pandas as pd

from run_all.main_preprocess import load_data, add_features
from utilities.utilities import get_latest_file

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Preprocess

In [ ]:
%%time
# Load original sources and combine to one DataFrame
# df_dataset_WMO, df_households = load_data()
df_dataset_WMO = load_data(region='gemeente')

In [ ]:
df_dataset_WMO.shape

## Conclusion: Wijk doet het prima, gemeente levert leeg df op. Uitzoeken hoe dat kan.....

In [ ]:
df_households.head()

In [ ]:
df_dataset_WMO = pd.merge(df_dataset_WMO, df_households, how='inner', left_on=['gemeentenaam', 'interval'],
                          right_on=['gemeentenaam', 'interval'])
df_dataset_WMO = df_dataset_WMO.set_index(['codering_regio', 'interval'])

In [ ]:
df_dataset_WMO = df_dataset_WMO.head()

In [ ]:
df_dataset_WMO

In [ ]:
df_wijk = get_and_combine_cbs_tables(dict_tables=settings.WIJK_TABLES,
                                     double_trouble_colnames=settings.DOUBLETROUBLECOLNAMES_WIJK,
                                     url=settings.CBS_OPEN_URL)
df_wijk_sub = rename_and_subset_cols(df=df_wijk,
                                     dict_rename=settings.DICT_WIJK_COLS_RENAMED,
                                     list_cols=['id', 'wijkenenbuurten', 'soortregio',
                                                'indelingswijzigingwijkenenbuurten'],
                                     include=False)
df_wijk_sub['codering_regio'] = df_wijk_sub['codering_regio'].str.strip()
df_wijk_sub['gemeentenaam'] = df_wijk_sub['gemeentenaam'].str.strip()
df_wijk_sub['perioden'] = df_wijk_sub['interval'] # just to apply function
df_pc_gem = df_wijk_sub[df_wijk_sub.codering_regio.str.startswith('BU', na=False)][
    ['codering_regio', 'interval', 'gemeentenaam', 'meestvoorkomendepostcode']].copy()

In [ ]:
df_pc_gem.sample(5)

In [ ]:
df_pc_gem = df_pc_gem.rename(columns={'meestvoorkomendepostcode':'postcode'})
df_pc_gem['postcode'] = df_pc_gem['postcode'].str.strip()
df_pc_gem = df_pc_gem.set_index(['postcode', 'interval'])
df_pc_gem.sample(5)

In [ ]:
df_pc_gem.index

In [ ]:
df_pc_gem = df_pc_gem.rename({'meestvoorkomendepostcode':'postcode'})
df_pc_gem = df_pc_gem.set_index(['postcode', 'interval'])

In [ ]:
import src.settings as settings
from src.preprocess.preprocess import get_and_combine_cbs_tables, rename_and_subset_cols, \
    get_region_period_spec_val_subtable, downcast_variables_dataframe
import numpy as np

In [ ]:
df_households = get_and_combine_cbs_tables(dict_tables=settings.DICT_POSITIE_HUISHOUDEN, url=settings.CBS_OPEN_URL)
df_households.rename(columns={'perioden': 'interval'})
indexNames = df_households[(df_households['postcode'] == 'Nederland') | (df_households['postcode'] == 'Niet in te delen')].index
df_households.drop(indexNames , inplace=True)
for col in ['geslacht', 'positieinhethuishouden']:
    df_households[col] = df_households[col].str.lower().str.replace(" ", "_")
df_households['positiehuishouden'] = df_households['positieinhethuishouden'] + '_' + df_households['geslacht']
df_households = df_households[['bevolking', 'positiehuishouden', 'postcode', 'interval']]
df_households = pd.pivot_table(data=df_households, values='bevolking', index=['postcode','interval'], columns=['positiehuishouden'], aggfunc=np.sum).reset_index() 
df_households = df_households.set_index(['postcode', 'interval'])
df_households = pd.merge(df_households, df_pc_gem, how='inner', left_index=True, right_index=True).reset_index()
df_households = df_households.groupby(by=["codering_regio"]).sum().reset_index()
df_households = df_households.set_index(['codering_regio', 'interval'])

In [ ]:
df_households.head(5)

In [ ]:
df_householdss = df_households.copy()

In [ ]:
df_households.rename(columns={'perioden': 'interval'})
for col in ['geslacht', 'positieinhethuishouden']:
    df_householdss[col] = df_householdss[col].str.lower().str.replace(" ", "_")
df_householdss['positiehuishouden'] = df_householdss['positieinhethuishouden'] + '_' + df_householdss['geslacht']
df_householdss = df_householdss[['bevolking', 'positiehuishouden', 'postcode', 'interval']]
df_householdss = pd.pivot_table(data=df_householdss, values='bevolking', index=['postcode','interval'], columns=['positiehuishouden'], aggfunc=np.sum).reset_index() 
df_householdss = df_householdss.set_index(['postcode', 'interval'])
df_householdss = pd.merge(df_householdss, df_pc_gem, how='inner', left_index=True, right_index=True).reset_index()
df_householdss = df_householdss.groupby(by=['gemeentenaam', 'interval']).sum().reset_index()
# df_householdss = df_householdss.set_index(['codering_regio', 'interval'])

In [ ]:
df_householdss.head()

In [ ]:
df_dataset_WMO2 = pd.merge(df_dataset_WMO, df_householdss, how='inner', left_on=['gemeentenaam', 'interval'], right_on=['gemeentenaam', 'interval'])

In [ ]:
df_dataset_WMO2.head()

In [ ]:
df_dataset_WMO2.shape

In [ ]:
df_householdss.head()

In [ ]:
df_households.positieinhethuishouden.unique()

In [ ]:
df_households.geslacht.unique()

In [ ]:
indexNames

In [ ]:
'84721NED'

In [ ]:
import cbsodata
df_sub = pd.DataFrame(cbsodata.get_data('84721NED', catalog_url='opendata.cbs.nl'))

In [ ]:
df_sub

In [ ]:
df_wmo = pd.DataFrame(cbsodata.get_data('84583NED', catalog_url='opendata.cbs.nl'))

In [ ]:
df_wmo.head()

In [ ]:
df_wmo = df_wmo[df_wmo.Codering_3.str.startswith('BU', na=False)]

In [ ]:
df_wmo.MeestVoorkomendePostcode_103.nunique()

In [ ]:
df_wmo[['Gemeentenaam_1', 'Codering_3', 'MeestVoorkomendePostcode_103']]

In [ ]:
df_wmo['Codering_3'].unique()

In [ ]:
%%time
# Feature engineering to get more features
df_dataset_WMO_with_features = add_features(df_dataset_WMO)

## Optional: Write temporary result

In [ ]:
# suffix_datetime = datetime.strftime(datetime.now(), format='%Y%m%d%H%M')

# df_dataset_WMO_with_features.to_parquet(f'../../data/df_preprocess_WMO_{suffix_datetime}.parquet.gzip',
#               compression='gzip')

# Train

## Optional: Load previous dataset

In [ ]:
# ## HARDCODED
# datapath = '../../data/'
# filename = 'df_preprocess_WMO_202103211137.parquet.gzip'
# df = pd.read_parquet(datapath + filename)

# ## SELECT LAST FILE
# datapath = '../../data/'
# df = get_latest_file(filename_str_contains='df_preprocess_WMO_', datapath=datapath, filetype='parquet')

## Train model
Code will be added later